In [ ]:
import kagglehub

path = kagglehub.dataset_download("kaggle/meta-kaggle")
print("Path to dataset files:", path)

In [6]:
# sournce: https://www.kaggle.com/code/sahidvelji/meet-the-grandmasters 
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import time
import re
import cssutils
import json
import matplotlib.pyplot as plt

from IPython.display import display, HTML
        
KAGGLE_BASE_URL = 'https://kaggle.com/'
TWITTER_BASE_URL = 'https://twitter.com/'
GITHUB_BASE_URL = 'https://github.com/'
DataREPO = './data/1737/'
#TOP_N = 10

# modified
import os
def beep():
    os.system('afplay /System/Library/Sounds/Ping.aiff')

import pickle

In [7]:
def display_html(df, cols=None, num_rows=0):
    """Display columns cols and num_rows rows of the data 
    frame df in HTML.
    """
    
    if num_rows != 0:
        df_to_display = df.head(num_rows)
    else:
        df_to_display = df
    
    df_html = df_to_display.to_html(columns=cols, index=False, na_rep='',
                              escape=False, render_links=True)
    display(HTML(df_html))

In [8]:
# Users
users = pd.read_csv(DataREPO+'Users.csv', encoding='utf-8')
user_ach = pd.read_csv(DataREPO+'UserAchievements.csv', encoding='utf-8')

beep() # 

string = f"There are {users.shape[0]} data in users."
display(HTML(string))
string = f"There are {users.shape[0]} data in user_ach."
display(HTML(string))


In [10]:
# PerformanceTier: 2(expert),3(master),4(grandmaster)
pftier = 4
gms = (users[users['PerformanceTier'] == pftier]
        .copy()
        .reset_index(drop=True)
        .drop(columns=['PerformanceTier'])
      )
string = f"There are currently {gms.shape[0]} Grandmasters."
display(HTML(string))

gms_ach = (user_ach[user_ach['Tier'] == pftier]
            .copy()
            .reset_index(drop=True)
            .drop(columns=['Tier','TierAchievementDate','Points','CurrentRanking','HighestRanking','TotalGold','TotalSilver','TotalBronze'])
          )
string = f"There are currently {gms_ach.shape[0]} Grandmasters."
display(HTML(string))




In [ ]:
#print(gms.head(3))
uid = gms_ach.UserId.unique()
print(len(uid))
svfile = './gmasters.xlsx'

df_GM = pd.DataFrame(columns= ['id','username','displayname','country','category'])

kgteam = 0
luid = len(uid)
for ii in range(0,luid):
    id = uid[ii]
    udf = users[users["Id"]==id]
    pt = udf.PerformanceTier.iloc[0]
    un = udf.UserName.iloc[0]
    dn = udf.DisplayName.iloc[0]
    cn = udf.Country.iloc[0]
    if pt == 5:
        kgteam += 1
    pd_tmp = gms_ach[gms_ach["UserId"]==id]
    df_GM.loc[ii] = [id, un, dn, cn, pd_tmp.AchievementType.iloc[0:len(pd_tmp)].tolist()]
    
print(kgteam)
print(df_GM)
print(df_GM.shape)

df_GM.to_excel(svfile, sheet_name='GrandMasters',\
                columns = ['id','username','displayname','country','category'])

beep() # modified

In [ ]:
pklfl = './gmaster.pkl'
svfile = './gmasters.xlsx'
sltime = 5

def fetch_profile(luid,stind,df_GM):
    capabilities = {
        "browserName": "MicrosoftEdge",
        "ms:edgeOptions": {
            "args": ["--headless"]
        }
    }
    for ii in range(stind,luid):
        print(ii+1,'/',luid)
        username = df_GM.loc[ii,'username']
        displayname = df_GM.loc[ii,'displayname']
        print(displayname)

        profile_url = f'{KAGGLE_BASE_URL}{username}'
        df_GM.loc[ii,'ProfileUrl'] = profile_url
        print(profile_url)

        fname = "./" + f"{ii:04}" + "_" + username + ".txt"
        if os.path.isfile(fname):
            with open(fname, encoding='utf-8') as f:
                bs = BeautifulSoup(f.read(), 'html.parser')
        else:
            driver = webdriver.Edge(executable_path='./msedgedriver', capabilities=capabilities) # locate appropriate webdriver in the executable-file directory
            driver.get(profile_url)
            time.sleep(sltime)
            bs = BeautifulSoup(driver.page_source.encode('utf-8'), 'html.parser')
            with open(fname, mode='w') as f:
                f.write(str(bs.prettify()))

        tmpstl = bs.find("div", {"class":"sc-iihIvb bUwKTg"}) # note that the class will be changed with a few days
        if not tmpstl:
            continue

        tmp = tmpstl['style']
        df_GM.loc[ii,'ImageUrl'] = cssutils.parseStyle(tmp)['background-image']
        
        tmplst = bs.find("ul",{"class":"sc-eFKCff gwCSOQ"}) # note that the class will be changed with a few days
        lis = tmplst.findAll("li")
        for ll in lis:
            tmpspan = ll.find("span").get_text().strip() #work, pin_drop, ... 
            tmpp = ll.find("p").get_text().strip()
            if re.search("work",tmpspan):
                tmplst = re.split(' at ',tmpp)
                if len(tmplst) == 0:
                    df_GM.loc[ii, 'Occupation'] = tmpp
                    print(tmpp)
                if len(tmplst) == 1:
                    print(tmpp)
                elif len(tmplst) == 2:
                    df_GM.loc[ii, 'Occupation'] = tmplst[0]
                    df_GM.loc[ii, 'Organization'] = tmplst[1]
                else:
                    print(tmpp)
            elif re.search("pin_drop",tmpspan):
                tmplst = re.split(', ',tmpp)
                if len(tmplst) == 0:
                    df_GM.loc[ii, 'Country'] = tmpp
                elif len(tmplst) == 1:
                    df_GM.loc[ii, 'Country'] = tmplst[0]
                    print(tmplst)
                elif len(tmplst) == 2:
                    df_GM.loc[ii, 'City'] = tmplst[0]
                    df_GM.loc[ii, 'Country'] = tmplst[1]
                elif len(tmplst) == 3:
                    df_GM.loc[ii, 'Region'] = tmplst[0]
                    df_GM.loc[ii, 'City'] = tmplst[1]
                    df_GM.loc[ii, 'Country'] = tmplst[2]
                else:
                    print(tmp.get_text())

        tmplst = bs.find("div",{"class":"MuiTabs-flexContainer css-k008qs"})
        alst = tmplst.findAll("a")
        for aa in alst:
            tmp = aa.find("span").get_text().strip()
            if re.search("Competitions",tmp):
                df_GM.loc[ii, 'NumCompetitions'] = re.search("Competitions (\d*)",tmp)[0]
            elif re.search("Datasets",tmp):
                df_GM.loc[ii, 'NumDatasets'] = re.search("Datasets (\d*)",tmp)[0]
            elif re.search("Code",tmp):
                df_GM.loc[ii, 'NumCodes'] = re.search("Code (\d*)",tmp)[0]
            elif re.search("Discussion",tmp):
                df_GM.loc[ii, 'NumDiscussions'] = re.search("Discussion (\d*)",tmp)[0]
            elif re.search("Followers",tmp):
                df_GM.loc[ii, 'NumFollowers'] = re.search("Followers (\d*)",tmp)[0]
            elif re.search("Following",tmp):
                df_GM.loc[ii, 'NumFollowing'] = re.search("Following (\d*)",tmp)[0]


In [ ]:
fetch_profile(luid,0,df_GM)
df_GM.to_excel(svfile, sheet_name='GrandMasters',\
                columns = ['id','username','displayname',
                           'ProfileUrl',
                           'country','category',
                           'Occupation', 'Organization'])
beep() # modified

1 / 619
José H. Solórzano
https://kaggle.com/solorzano
Machine Learning Engineer
2 / 619
Bluefool
https://kaggle.com/domcastro
['United Kingdom']
3 / 619
Charlie Tang
https://kaggle.com/neocortex
4 / 619
Alexander D'yakonov
https://kaggle.com/dyakonov
5 / 619
Naokazu Mizuta
https://kaggle.com/naokazumizuta
6 / 619
Jeremy Howard
https://kaggle.com/jhoward
7 / 619
Will Cukierski
https://kaggle.com/wcukierski
['United States']
8 / 619
Eu Jin Lok
https://kaggle.com/ejlok1
9 / 619
B Yang
https://kaggle.com/byang1
10 / 619
Sergey Yurgenson
https://kaggle.com/ccccat
11 / 619
Vivek Sharma
https://kaggle.com/viveksha
12 / 619
jontix
https://kaggle.com/jontix
13 / 619
Stefan Stefanov
https://kaggle.com/stefanstefanov
14 / 619
Kohei
https://kaggle.com/confirm
15 / 619
Jason Tigg
https://kaggle.com/jasontigg
16 / 619
Glen Koundry
https://kaggle.com/gkoundry
17 / 619
Stefan Henß
https://kaggle.com/shenss
18 / 619
Neil Schneider, FSA
https://kaggle.com/nschneider
19 / 619
Owen
https://kaggle.com/owe